# 1. Logistic Regression

In [122]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
# from config import db_password
from sqlalchemy import inspect
import matplotlib.pyplot as plt
import pandas as pd

business_df = pd.read_csv("../../../Data/business_census_merged_dataset.csv")

# Categorizing restaurants based on stars ratings
business_df["Category"] = pd.cut(business_df["Stars_Rating"],bins=[0.9,3,5],
                                 labels=["Lower Performance", "Higher Performance"])


def changeStatus(status):
    if status == "Lower Performance":
        return 0
    else:
        return 1
    
business_df['Category_Encoded'] = business_df["Category"].apply(changeStatus)
business_df["Category_Encoded"] = pd.to_numeric(business_df["Category_Encoded"])

business_df["Median_Income(dollars)"] = pd.to_numeric(business_df["Median_Income(dollars)"], errors='coerce')
business_df["Mean_Income(dollars)"] = pd.to_numeric(business_df["Mean_Income(dollars)"], errors='coerce')

business_df = business_df.dropna(subset=['Median_Income(dollars)'])

In [123]:
y = business_df["Category"]
from collections import Counter
Counter(y)

Counter({'Higher Performance': 18905, 'Lower Performance': 5834})

In [124]:
business_df.columns

Index(['Restaurant_ID', 'Restaurants_Name', 'Address', 'City', 'State',
       'Postal_Code', 'Latitude', 'Longitude', 'Stars_Rating', 'Review_Count',
       'Restaurants_Delivery', 'Outdoor_Seating', 'Accepts_CreditCards',
       'Price_Range', 'Alcohol', 'Good_For_Kids', 'Reservations',
       'Restaurants_TakeOut', 'WiFi', 'Good_For_Groups',
       'Wheelchair_Accessible', 'Happy_Hour', 'Noise_Level',
       'Dietary_Restrictions', 'Total_Estimate_Households_per_Zip',
       'Total_Estimate_Married-couple_Family_households',
       'Total_Estimate_Nonfamily_households', 'Median_Income(dollars)',
       'Mean_Income(dollars)', 'Category', 'Category_Encoded'],
      dtype='object')

In [125]:
# Define features set
X = business_df[['Review_Count', 'Restaurants_Delivery', 'Outdoor_Seating',
       'Accepts_CreditCards', 'Price_Range', 'Alcohol', 'Good_For_Kids',
       'Reservations', 'Restaurants_TakeOut', 'WiFi', 'Good_For_Groups',
       'Wheelchair_Accessible', 'Happy_Hour', 'Noise_Level',
       'Dietary_Restrictions',
                'Total_Estimate_Married-couple_Family_households',
       'Total_Estimate_Nonfamily_households',
                'Median_Income(dollars)','Total_Estimate_Households_per_Zip']]

In [126]:
# Define the target
y = business_df["Category_Encoded"]
y

0        1
1        1
2        1
3        0
4        1
        ..
24734    1
24735    1
24736    0
24737    1
24738    0
Name: Category_Encoded, Length: 24739, dtype: int64

In [127]:
# Split the model into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y, 
                                                   random_state=1, 
                                                    stratify=y)

In [128]:
# Instantiate a Logistic Regression Model
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver="lbfgs", random_state=1)
model

LogisticRegression(random_state=1)

In [129]:
# Train the model
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [130]:
# Make predictions
y_pred = model.predict(X_test)

In [131]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.76410670978173

In [132]:
# Evaluate the Model

from sklearn.metrics import accuracy_score
y_pred_train = model.predict(X_train)
accuracy_score(y_train, y_pred_train)

0.7642017893715641

In [133]:
# Calculate the confusion matrix 
from sklearn.metrics import confusion_matrix, classification_report

matrix=confusion_matrix(y_test, y_pred)
print(matrix)

[[   0 1459]
 [   0 4726]]


In [135]:
report=classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1459
           1       0.76      1.00      0.87      4726

    accuracy                           0.76      6185
   macro avg       0.38      0.50      0.43      6185
weighted avg       0.58      0.76      0.66      6185



/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

# Logistic Regression with Oversampling

In [136]:
from collections import Counter
Counter(y)

Counter({1: 18905, 0: 5834})

It seems like the classes are unbalanced, which might bias the results toward the majority classes. 

# Naive Random Oversampling


In random oversampling, instances of the minority class are randomly selected and added to the training set until the majority and minority classes are balanced.  

In [137]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)

X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({0: 14179, 1: 14179})

In [138]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [139]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6024690387592357

In [140]:
y_pred_train = model.predict(X_train)
balanced_accuracy_score(y_train, y_pred_train)

0.6194546462071675

In [141]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[1085,  374],
       [2546, 2180]])

In [142]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.30      0.74      0.46      0.43      0.59      0.35      1459
          1       0.85      0.46      0.74      0.60      0.59      0.33      4726

avg / total       0.72      0.53      0.68      0.56      0.59      0.34      6185



# SMOTE Oversampling

In SMOTE, like naive random oversampling, the size of the minority is increased. The key difference with random oversampling is how the minority class is increased in size. 

However, in SMOTE, new instances are interpolated. That is, for an instance from the minority class, new values are generated based on its distance from its neighbors.

**Random oversampling draws from existing observations, whereas SMOTE generates synthetic observations**.

In [143]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy="auto").fit_resample(X_train, y_train)

from collections import Counter

Counter(y_resampled)

Counter({0: 14179, 1: 14179})

In [144]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [146]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6029358104453018

In [147]:
y_pred_train = model.predict(X_train)
balanced_accuracy_score(y_train, y_pred_train)

0.6203868739483945

In [148]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[1070,  389],
       [2493, 2233]])

In [149]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.30      0.73      0.47      0.43      0.59      0.36      1459
          1       0.85      0.47      0.73      0.61      0.59      0.34      4726

avg / total       0.72      0.53      0.67      0.56      0.59      0.34      6185



# Cluster Centroids Undersampling

Akin to SMOTE, the algorithm identifies clusters of the majority class, then generates synthetic data points, called centroids, that are representative of the clusters. The majority class is then undersampled down to the size of the minority class. 

In [150]:
# Resample the data using the ClusterCentroids resampler

from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

from collections import Counter

Counter(y_resampled)

Counter({0: 4375, 1: 4375})

In [151]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [152]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [153]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5980266224467509

In [154]:
y_pred_train = model.predict(X_train)
accuracy_score(y_train, y_pred_train)

0.47606985016707987

In [155]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[1223,  236],
       [3035, 1691]])

In [156]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.29      0.84      0.36      0.43      0.55      0.31      1459
          1       0.88      0.36      0.84      0.51      0.55      0.29      4726

avg / total       0.74      0.47      0.72      0.49      0.55      0.29      6185



# Combination of Over and Under-Sampling

A downside of oversampling with SMOTE is its reliance on the immediate neighbors of a data point. Because the algorithm doesn't see the overall distribution of data, the new data it creates can be heavily influenced by outliers. 

A downside of undersampling is that it involves loss of data and is not an option when the dataset is small. 

SMOTEEN combines the SMOTE and Edited Nearest Neighbors (ENN) algorithms. It includes the following steps: 
1. Oversample the minority class with SMOTE.
2. Clean the resulting data with an undersampling strategy. If the two of the nearest neighbors of a data point belong to different classes, the data point is dropped.

In [157]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 7176, 1: 5616})

In [158]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [159]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6029112282483815

In [160]:
y_pred_train = model.predict(X_train)
balanced_accuracy_score(y_train, y_pred_train)

0.61665957502544

In [161]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[1161,  298],
       [2788, 1938]])

In [162]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.29      0.80      0.41      0.43      0.57      0.34      1459
          1       0.87      0.41      0.80      0.56      0.57      0.31      4726

avg / total       0.73      0.50      0.70      0.53      0.57      0.32      6185

